# Two-Stage Optimization

This notebook demonstrates a **two-stage optimization** approach:

1. **Stage 1: Sizing** - Optimize component sizes using downsampled data
2. **Stage 2: Dispatch** - Optimize operation with fixed sizes at full resolution

This technique is useful for:
- **Large models** where full optimization is computationally expensive
- **Faster development** iterations
- **Investment planning** where sizing decisions can be approximated first

We'll compare this approach to a combined (single-stage) optimization.

## Setup

In [ ]:
import logging
import pathlib
import timeit

import pandas as pd
import xarray as xr

import flixopt as fx

logger = logging.getLogger('flixopt')
fx.CONFIG.notebook()

## Load Data

In [ ]:
# Load time series data
data_path = pathlib.Path('..') / '..' / 'examples' / 'resources' / 'Zeitreihen2020.csv'
data_import = pd.read_csv(data_path, index_col=0).sort_index()

# Use first 500 timesteps for this example
filtered_data = data_import[:500]
filtered_data.index = pd.to_datetime(filtered_data.index)
timesteps = filtered_data.index

print(f'Time range: {timesteps[0]} to {timesteps[-1]}')
print(f'Number of timesteps: {len(timesteps)}')

In [ ]:
# Extract time series
electricity_demand = filtered_data['P_Netz/MW'].to_numpy()
heat_demand = filtered_data['Q_Netz/MW'].to_numpy()
electricity_price = filtered_data['Strompr.€/MWh'].to_numpy()
gas_price = filtered_data['Gaspr.€/MWh'].to_numpy()

## Build FlowSystem with Investment Options

This system includes **InvestParameters** for component sizing:

In [ ]:
flow_system = fx.FlowSystem(timesteps)

# Add all components in one call
flow_system.add_elements(
    # Buses
    fx.Bus('Strom', carrier='electricity'),
    fx.Bus('Fernwärme', carrier='heat'),
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Kohle', carrier='fuel'),
    # Effects
    fx.Effect('costs', '€', 'Kosten', is_standard=True, is_objective=True),
    fx.Effect('CO2', 'kg', 'CO2_e-Emissionen'),
    fx.Effect('PE', 'kWh_PE', 'Primärenergie'),
    # Gas Boiler with investment optimization
    fx.linear_converters.Boiler(
        'Kessel',
        thermal_efficiency=0.85,
        thermal_flow=fx.Flow(label='Q_th', bus='Fernwärme'),
        fuel_flow=fx.Flow(
            label='Q_fu',
            bus='Gas',
            size=fx.InvestParameters(
                effects_of_investment_per_size={'costs': 1_000},  # 1000 €/kW
                minimum_size=10,
                maximum_size=600,
            ),
            relative_minimum=0.2,
            previous_flow_rate=20,
            status_parameters=fx.StatusParameters(effects_per_startup=300),
        ),
    ),
    # CHP with investment optimization
    fx.linear_converters.CHP(
        'BHKW2',
        thermal_efficiency=0.58,
        electrical_efficiency=0.22,
        status_parameters=fx.StatusParameters(effects_per_startup=1_000, min_uptime=10, min_downtime=10),
        electrical_flow=fx.Flow('P_el', bus='Strom'),
        thermal_flow=fx.Flow('Q_th', bus='Fernwärme'),
        fuel_flow=fx.Flow(
            'Q_fu',
            bus='Kohle',
            size=fx.InvestParameters(
                effects_of_investment_per_size={'costs': 3_000},  # 3000 €/kW
                minimum_size=10,
                maximum_size=500,
            ),
            relative_minimum=0.3,
            previous_flow_rate=100,
        ),
    ),
    # Storage with investment optimization
    fx.Storage(
        'Speicher',
        capacity_in_flow_hours=fx.InvestParameters(
            minimum_size=10,
            maximum_size=1000,
            effects_of_investment_per_size={'costs': 60},  # 60 €/kWh
        ),
        initial_charge_state='equals_final',  # Cyclic operation
        eta_charge=1,
        eta_discharge=1,
        relative_loss_per_hour=0.001,
        prevent_simultaneous_charge_and_discharge=True,
        charging=fx.Flow('Q_th_load', size=200, bus='Fernwärme'),
        discharging=fx.Flow('Q_th_unload', size=200, bus='Fernwärme'),
    ),
    # Sinks and Sources
    fx.Sink('Wärmelast', inputs=[fx.Flow('Q_th_Last', bus='Fernwärme', size=1, fixed_relative_profile=heat_demand)]),
    fx.Source(
        'Gastarif',
        outputs=[fx.Flow('Q_Gas', bus='Gas', size=1000, effects_per_flow_hour={'costs': gas_price, 'CO2': 0.3})],
    ),
    fx.Source(
        'Kohletarif',
        outputs=[fx.Flow('Q_Kohle', bus='Kohle', size=1000, effects_per_flow_hour={'costs': 4.6, 'CO2': 0.3})],
    ),
    fx.Source(
        'Einspeisung',
        outputs=[
            fx.Flow(
                'P_el', bus='Strom', size=1000, effects_per_flow_hour={'costs': electricity_price + 0.5, 'CO2': 0.3}
            )
        ],
    ),
    fx.Sink('Stromlast', inputs=[fx.Flow('P_el_Last', bus='Strom', size=1, fixed_relative_profile=electricity_demand)]),
    fx.Source(
        'Stromtarif',
        outputs=[
            fx.Flow('P_el', bus='Strom', size=1000, effects_per_flow_hour={'costs': electricity_price, 'CO2': 0.3})
        ],
    ),
)

## Two-Stage Optimization

### Stage 1: Sizing with Downsampled Data

We use `resample('2h')` to reduce the problem size by half. This speeds up the investment optimization significantly:

In [ ]:
start = timeit.default_timer()

# Create downsampled version (2-hour resolution)
calculation_sizing = fx.Optimization('Sizing', flow_system.resample('2h'))
calculation_sizing.do_modeling()
calculation_sizing.solve(fx.solvers.HighsSolver(0.1 / 100, 60))

timer_sizing = timeit.default_timer() - start
print(f'\nSizing optimization completed in {timer_sizing:.2f} seconds')

### Check Optimized Sizes

In [ ]:
print('Optimized sizes from Stage 1:')
calculation_sizing.flow_system.statistics.sizes

### Stage 2: Dispatch with Fixed Sizes

Now optimize the dispatch at full resolution, but with sizes fixed from Stage 1:

In [ ]:
start = timeit.default_timer()

# Create dispatch optimization with original (full resolution) flow_system
calculation_dispatch = fx.Optimization('Dispatch', flow_system)
calculation_dispatch.do_modeling()

# Fix sizes from sizing optimization
calculation_dispatch.fix_sizes(calculation_sizing.flow_system.solution)

calculation_dispatch.solve(fx.solvers.HighsSolver(0.1 / 100, 60))

timer_dispatch = timeit.default_timer() - start
print(f'\nDispatch optimization completed in {timer_dispatch:.2f} seconds')

### Verify Size Consistency

In [ ]:
dispatch_sizes = calculation_dispatch.flow_system.statistics.sizes
sizing_sizes = calculation_sizing.flow_system.statistics.sizes

if (dispatch_sizes.round(5).to_dataarray() == sizing_sizes.round(5).to_dataarray()).all().item():
    print('Sizes were correctly transferred from Stage 1 to Stage 2')
else:
    print('WARNING: Sizes mismatch!')

## Combined (Single-Stage) Optimization

For comparison, let's also solve the full problem in one go:

In [ ]:
start = timeit.default_timer()

calculation_combined = fx.Optimization('Combined', flow_system)
calculation_combined.do_modeling()
calculation_combined.solve(fx.solvers.HighsSolver(0.1 / 100, 600))  # More time for full problem

timer_combined = timeit.default_timer() - start
print(f'\nCombined optimization completed in {timer_combined:.2f} seconds')

## Compare Results

### Build Comparison Dataset

In [ ]:
comparison = xr.concat(
    [calculation_combined.flow_system.solution, calculation_dispatch.flow_system.solution], dim='mode'
).assign_coords(mode=['Combined', 'Two-stage'])

# Add timing information
comparison['Duration [s]'] = xr.DataArray([timer_combined, timer_sizing + timer_dispatch], dims='mode')

### Key Metrics Comparison

In [ ]:
comparison_main = comparison[
    [
        'Duration [s]',
        'costs',
        'costs(periodic)',
        'costs(temporal)',
        'BHKW2(Q_fu)|size',
        'Kessel(Q_fu)|size',
        'Speicher|size',
    ]
]

# Calculate percentage difference
diff = (
    (comparison_main.sel(mode='Two-stage') - comparison_main.sel(mode='Combined'))
    / comparison_main.sel(mode='Combined')
    * 100
).assign_coords(mode='Diff [%]')

comparison_main = xr.concat([comparison_main, diff], dim='mode')

print(comparison_main.to_pandas().T.round(2))

## Analysis

The comparison shows:

| Metric | Combined | Two-Stage | Difference |
|--------|----------|-----------|------------|
| Duration | Longer | **Faster** | Significant speedup |
| Total Costs | Optimal | Near-optimal | Small suboptimality |
| Sizes | Optimal | Approximate | May differ slightly |

## When to Use Two-Stage Optimization

**Recommended for:**
- Large models (many timesteps, components)
- Development and debugging phases
- Screening studies where many configurations are tested
- Problems where sizing decisions are less sensitive to exact dispatch

**Not recommended for:**
- Small problems (overhead not worth it)
- Problems where sizing and dispatch are tightly coupled
- When exact optimal solutions are required

## Summary

This example demonstrated:

1. **`flow_system.resample()`** - Downsample time series for faster computation
2. **`optimization.fix_sizes()`** - Transfer investment decisions between optimizations
3. **Two-stage workflow** - Separate sizing and dispatch for better scalability
4. **Trade-off analysis** - Speed vs. optimality comparison

The two-stage approach provides a practical way to handle large-scale energy system models!